### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [46]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from math import sqrt
import plotly.graph_objects as go

In [6]:
df = pd.read_csv("localdataset/data.csv")
df["date"] = pd.to_datetime(df["date"])
df.dropna(inplace=True)
df["mese_numerico"] = df["date"].dt.month + 12*(df["date"].dt.year-df["date"].dt.year.min())

In [42]:
def rmse(y, pred):
    y = np.asarray(y)
    if len(y)!=len(pred):
        raise ValueError("Arrays are different length")
    return sqrt(sum([(y[i]-pred[i])**2 for i in range(len(y))])/len(y))

In [ ]:
def plot_reg(x, y, degree):
    coefficients = np.polyfit(x, y, degree)
    poly_fun = np.poly1d(coefficients)
    domain = np.linspace(x.min(), x.max(), 100)
    y_pred = poly_fun(domain)
    y_pred_low_domain = poly_fun(x)
    print(f"rmse: {rmse(y, y_pred_low_domain)}")
    # plt.plot(domain, y_pred)
    # plt.xticks(x[::30], labels=df["date"].iloc[::30].dt.strftime("%Y-%m"))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["mese_numerico"], y=df["passengers"],
                         mode='markers', name='Dati'))
    fig.add_trace(go.Scatter(x=domain, y=y_pred,
                         mode='lines', name=f'Regressione grado {degree}'))
    fig.update_layout(title="Regressione",
                  xaxis_title="Data",
                  yaxis_title="Passeggeri",

                  template="plotly_white")
    tickvals = df["date"].iloc[::10]
    ticktext = df["date"].iloc[::10].dt.strftime("%Y-%m")
    fig.update_xaxes(
        tickvals=tickvals,       # Dove posizionare i tick
        ticktext=ticktext        # Quali etichette mostrare
    )
    fig.show()

In [ ]:
# plt.plot(df["mese_numerico"], df["passengers"], "o", alpha=0.8)
plot_reg(df["mese_numerico"], df["passengers"], 2)

rmse: 44.45027656808772


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva